<a href="https://colab.research.google.com/github/thiago4int/dss/blob/main/cobb_douglas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assumptions das máquinas (VMs e DBs)
vm_sem_commit_preco = 212
vm_com_commit_preco = 105
db_sem_commit_preco = 420
db_com_commit_preco = 210

# Especificações: 8 vCPUs e 32 GB de RAM para todas as VMs e DBs

# Função Cobb-Douglas para calcular a produção com base nas VMs e DBs
def funcao_cobb_douglas(qtd_vms, qtd_dbs):
    A = 1  # Constante de eficiência tecnológica
    alpha = 0.6  # Elasticidade das VMs (L intensive)
    beta = 0.4   # Elasticidade das DBs (K intensive)
    producao = A * (qtd_vms ** alpha) * (qtd_dbs ** beta)
    return producao

In [ ]:
# Função para calcular os custos com base na quantidade de instâncias e commit
def calcular_custos(qtd_vms, pct_vm_commit, qtd_dbs, pct_db_commit, desligar_nao_commitadas=False):
    # Cálculo da quantidade de VMs e DBs commitadas e não commitadas
    qtd_vms_commitadas = int((pct_vm_commit / 100) * qtd_vms)
    qtd_vms_nao_commitadas = qtd_vms - qtd_vms_commitadas

    qtd_dbs_commitadas = int((pct_db_commit / 100) * qtd_dbs)
    qtd_dbs_nao_commitadas = qtd_dbs - qtd_dbs_commitadas

    # Se pudermos desligar instâncias não commitadas, reduzimos o custo das não commitadas
    if desligar_nao_commitadas:
        qtd_vms_nao_commitadas = max(0, qtd_vms_nao_commitadas - int(0.5 * qtd_vms))  # Exemplo de desligar 50% das não commitadas
        qtd_dbs_nao_commitadas = max(0, qtd_dbs_nao_commitadas - int(0.5 * qtd_dbs))  # Exemplo de desligar 50% das não commitadas

    # Calcular o custo total para VMs
    custo_total_vms = (qtd_vms_commitadas * vm_com_commit_preco) + (qtd_vms_nao_commitadas * vm_sem_commit_preco)

    # Calcular o custo total para DBs
    custo_total_dbs = (qtd_dbs_commitadas * db_com_commit_preco) + (qtd_dbs_nao_commitadas * db_sem_commit_preco)

    return custo_total_vms, custo_total_dbs

# Função para calcular o custo de oportunidade
def calcular_custo_oportunidade(custo_total_commit, taxa_retorno, qtd_instancias_commitadas):
    """Calcula o custo de oportunidade baseado no custo commitado e na taxa de retorno (taxa livre de risco)."""
    custo_oportunidade = custo_total_commit * taxa_retorno * qtd_instancias_commitadas
    return custo_oportunidade

# Função para calcular o Índice de Sharpe ajustado
def calcular_sharpe(roi, risco, custo_oportunidade, taxa_livre_risco=0.1075):
    """Calcula o Índice de Sharpe ajustado considerando o ROI e o custo de oportunidade."""
    roi_ajustado = roi - custo_oportunidade  # Ajustar o ROI com base no custo de oportunidade
    sharpe_ratio = (roi_ajustado - taxa_livre_risco) / risco
    return sharpe_ratio

In [ ]:
# Função para calcular o ROI total e ajustar pelo Índice de Sharpe
def calcular_roi_ajustado(custo_total_vms, custo_total_dbs, receita_total, risco, taxa_retorno, qtd_vms_commitadas, qtd_dbs_commitadas):
    custo_total = custo_total_vms + custo_total_dbs
    fcf = receita_total - custo_total  # Fluxo de Caixa Livre (FCF)

    # Garantir que FCF seja positivo
    if fcf <= 0:
        return None, None  # Ignorar essa combinação se o FCF for negativo ou zero

    roi = fcf / custo_total  # ROI básico

    # Garantir que ROI seja positivo
    if roi <= 0:
        return None, None  # Ignorar essa combinação se o ROI for negativo ou zero

    # Calcular o custo de oportunidade total (considerando apenas instâncias commitadas)
    custo_oportunidade_vms = calcular_custo_oportunidade(custo_total_vms, taxa_retorno, qtd_vms_commitadas)
    custo_oportunidade_dbs = calcular_custo_oportunidade(custo_total_dbs, taxa_retorno, qtd_dbs_commitadas)

    # ROI ajustado pelo Sharpe
    roi_ajustado = calcular_sharpe(roi, risco, custo_oportunidade_vms + custo_oportunidade_dbs)

    return fcf, roi_ajustado

In [ ]:
# Função para simular vários cenários e encontrar o melhor mix de commit e não commit
def simular_melhor_mix(receita_total, risco, taxa_retorno, desligar_nao_commitadas=False):
    melhor_fcf = -np.inf
    melhor_roi = -np.inf
    melhor_mix = None
    melhor_budget = 0

    # Adicionamos mais verificações para garantir que há uma combinação válida
    encontrou_comb_validas = False

    # Simular budgets crescentes e diferentes porcentagens de commit para VMs e DBs
    for budget in range(1000000, 3000000, 100000):  # Orçamentos de $1M a $3M para garantir mais instâncias
        for pct_vm_commit in range(0, 101, 10):  # Incrementos de 10%
            for pct_db_commit in range(0, 101, 10):
                # Estimar quantas VMs e DBs podem ser provisionadas com o orçamento
                qtd_vms = budget // vm_sem_commit_preco
                qtd_dbs = budget // db_sem_commit_preco

                # Garantir que o número de instâncias seja positivo
                if qtd_vms <= 0 or qtd_dbs <= 0:
                    continue  # Pular esse budget se o número de instâncias for inválido

                encontrou_comb_validas = True  # Encontrou uma combinação válida, seguimos

                # Calcular os custos totais para VMs e DBs, agora considerando se podemos desligar instâncias não commitadas
                custo_total_vms, custo_total_dbs = calcular_custos(qtd_vms, pct_vm_commit, qtd_dbs, pct_db_commit, desligar_nao_commitadas=desligar_nao_commitadas)

                # Quantidade de instâncias commitadas
                qtd_vms_commitadas = int((pct_vm_commit / 100) * qtd_vms)
                qtd_dbs_commitadas = int((pct_db_commit / 100) * qtd_dbs)

                # Calcular a produção com base na função Cobb-Douglas
                receita_ajustada = funcao_cobb_douglas(qtd_vms, qtd_dbs)

                # Calcular o ROI ajustado pelo Sharpe
                fcf, roi_ajustado = calcular_roi_ajustado(custo_total_vms, custo_total_dbs, receita_ajustada, risco, taxa_retorno, qtd_vms_commitadas, qtd_dbs_commitadas)

                # Ignorar combinações com FCF ou ROI inválidos
                if fcf is None or roi_ajustado is None:
                    continue

                # Verificar se esse mix é o melhor até agora
                if roi_ajustado > melhor_roi or (roi_ajustado == melhor_roi and fcf > melhor_fcf):
                    melhor_roi = roi_ajustado
                    melhor_fcf = fcf
                    melhor_mix = (pct_vm_commit, pct_db_commit)
                    melhor_budget = budget

    # Verificar se houve combinação válida
    if not encontrou_comb_validas:
        print("Nenhuma combinação válida foi encontrada.")
        return

    if melhor_mix is None:
        print("Nenhuma combinação válida com FCF positivo foi encontrada.")
        return

    # Exibir o melhor mix e o budget encontrado
    print(f"Melhor Mix Encontrado:")
    print(f"% de VMs Commitadas: {melhor_mix[0]}%")
    print(f"% de DBs Commitadas: {melhor_mix[1]}%")
    print(f"Melhor Budget: ${melhor_budget}")
    print(f"Melhor Fluxo de Caixa Livre (FCF): ${melhor_fcf:.2f}")
    print(f"Melhor ROI Ajustado pelo Sharpe: {melhor_roi:.4f}")

    # Plotar os resultados
    if melhor_fcf > 0 and melhor_roi > 0:
        fig, ax = plt.subplots()
        ax.bar(['FCF', 'ROI Ajustado'], [melhor_fcf, melhor_roi], color=['blue', 'green'])
        ax.set_title('Melhor Mix: FCF e ROI Ajustado')
        ax.set_ylabel('Valores ($ ou %)')
        plt.show()

# Executar a simulação para encontrar o melhor mix com base na receita
simular_melhor_mix(receita_total=5000000, risco=0.15, taxa_retorno=0.1075, desligar_nao_commitadas=True)

Nenhuma combinação válida com FCF positivo foi encontrada.
